## Análise Dos Clusters Mapeados

In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.preprocessing import StandardScaler
from haversine import haversine
import plotly.express as px
import plotly.graph_objects as go

# Carregar os dados
df = pd.read_csv("../data/amostra_total.csv", sep=';')

# Normalização das coordenadas
scaler = StandardScaler()
coords = df[['LATITUDE', 'LONGITUDE']].values
coords_normalized = scaler.fit_transform(coords)

# Primeiro clustering com 350 clusters (leituristas)
n_clusters_leituristas = 350
kmeans_leituristas = MiniBatchKMeans(n_clusters=n_clusters_leituristas, random_state=88, batch_size=200, n_init='auto')
labels_leituristas = kmeans_leituristas.fit_predict(coords_normalized)
df.loc[:, 'Cluster_Leiturista'] = labels_leituristas

# Segundo clustering dentro de cada cluster de leituristas para 22 subclusters (dias de leitura)
n_clusters_dias = 22

# Plotar os clusters e subclusters
fig = go.Figure()

for leiturista in range(n_clusters_leituristas):  # Ajustar para range(n_clusters_leituristas) para leitura total
    cluster_data = df[df['Cluster_Leiturista'] == leiturista]
    if not cluster_data.empty:
        coords_cluster = cluster_data[['LATITUDE', 'LONGITUDE']].values
        coords_cluster_normalized = scaler.fit_transform(coords_cluster)
        kmeans_dias = MiniBatchKMeans(n_clusters=n_clusters_dias, random_state=88, batch_size=50, n_init='auto')
        labels_dias = kmeans_dias.fit_predict(coords_cluster_normalized)
        df.loc[cluster_data.index, 'Cluster_Dia'] = labels_dias

        # Atualizar cluster_data após adicionar Cluster_Dia
        cluster_data = df[df['Cluster_Leiturista'] == leiturista]

        # Adicionar círculos representando os subclusters
        for dia in range(n_clusters_dias):
            subcluster_data = cluster_data[cluster_data['Cluster_Dia'] == dia]
            if not subcluster_data.empty:
                centroid = subcluster_data[['LATITUDE', 'LONGITUDE']].mean().values
                radius = max([haversine(centroid, point) for point in subcluster_data[['LATITUDE', 'LONGITUDE']].values])

                fig.add_trace(go.Scattergeo(
                    lon=[centroid[1]],
                    lat=[centroid[0]],
                    mode='markers',
                    marker=dict(size=5, color='red', symbol='circle'),
                    name=f'Cluster {leiturista} Dia {dia}'
                ))

                fig.add_trace(go.Scattergeo(
                    lon=[centroid[1], centroid[1]],
                    lat=[centroid[0], centroid[0] + (radius / 110.574)],  # Convert km to degrees latitude
                    mode='lines',
                    line=dict(width=2, color='blue'),
                    showlegend=False
                ))

fig.update_layout(
    title='Clusters e Subclusters de Coordenadas Geográficas',
    geo=dict(
        scope='world',
        showland=True
    )
)

fig.show()


## Análise Dos Resultados Do Algoritmo De Clusterização

In [ ]:
import pandas as pd
from sklearn.cluster import MiniBatchKMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

def calculate_route_distance(cluster_data):
    # Suponha que cluster_data contém colunas 'LATITUDE' e 'LONGITUDE'
    points = list(zip(cluster_data['LATITUDE'], cluster_data['LONGITUDE']))
    total_distance = 0.0
    for i in range(1, len(points)):
        distance = haversine(points[i-1], points[i])  # Define a função haversine se necessário
        total_distance += distance
    return total_distance

# Função para calcular o tempo médio
def calculate_average_time(distances, speed_km_h):
    total_time = sum(distances) / speed_km_h  # Convertendo distância para tempo
    average_time = total_time / len(distances)
    return average_time

# Carregar dados
df = pd.read_csv("amostra_total.csv", sep=';')
scaler = StandardScaler()
coords = df[['LATITUDE', 'LONGITUDE']].values
coords_normalized = scaler.fit_transform(coords)

# Clustering
n_clusters = 350  # Ajuste conforme necessário
kmeans = MiniBatchKMeans(n_clusters=n_clusters, random_state=88, batch_size=200)
labels = kmeans.fit_predict(coords_normalized)
df['Cluster'] = labels

# Calcular a distância percorrida por cluster
cluster_distances = []
for cluster in range(n_clusters):
    cluster_data = df[df['Cluster'] == cluster]
    total_distance = calculate_route_distance(cluster_data)
    cluster_distances.append(total_distance)

# Velocidade média de um leiturista (km/h)
average_speed = 5  # Ajuste conforme necessário

# Calcular tempo médio de distância percorrida
average_time = calculate_average_time(cluster_distances, average_speed)

# Plotar gráfico de barras horizontal
plt.figure(figsize=(10, 2))
plt.barh(['Tempo Médio de Distância Percorrida'], [average_time])
plt.xlabel('Tempo (Minutos)')
plt.title('Tempo Médio de Distância Percorrida de Todos os Clusters')
plt.show()


In [ ]:
import pandas as pd
from sklearn.cluster import MiniBatchKMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

def calculate_route_distance(cluster_data):
    # Suponha que cluster_data contém colunas 'LATITUDE' e 'LONGITUDE'
    points = list(zip(cluster_data['LATITUDE'], cluster_data['LONGITUDE']))
    total_distance = 0.0
    for i in range(1, len(points)):
        distance = haversine(points[i-1], points[i])  # Define a função haversine se necessário
        total_distance += distance
    return total_distance

# Carregar dados
df = pd.read_csv("amostra_total.csv", sep=';')
scaler = StandardScaler()
coords = df[['LATITUDE', 'LONGITUDE']].values
coords_normalized = scaler.fit_transform(coords)

# Clustering
n_clusters = 350  # Ajuste conforme necessário
kmeans = MiniBatchKMeans(n_clusters=n_clusters, random_state=88, batch_size=200)
labels = kmeans.fit_predict(coords_normalized)
df['Cluster'] = labels

# Calcular a distância percorrida por cluster
cluster_distances = []
for cluster in range(n_clusters):
    cluster_data = df[df['Cluster'] == cluster]
    total_distance = calculate_route_distance(cluster_data)
    cluster_distances.append(total_distance)

# Calcular a média da distância percorrida
average_distance = sum(cluster_distances) / len(cluster_distances)

# Plotar gráfico de barras horizontal
plt.figure(figsize=(10, 2))
plt.barh(['Média de Distância Percorrida'], [average_distance], color = 'pink')
plt.xlabel('Distância no mês (km)')
plt.title('Média de Distância Percorrida por Cluster')
plt.show()


&emsp;&emsp;Com base nos resultados obtidos, o algoritmo processa toda a base de dados, juntamente com as métricas pré-definidas pelo parceiro. Os leituristas conseguem percorrer em média 6 horas por dia, o que resulta em mais de 300 km por mês. Dividindo isso por 22 dias, temos uma média de 13,6 km por dia. Com um ritmo de leitura de 5 minutos por ponto, e considerando que o tempo de execução da célula de clusterização é de cerca de 1 minuto, atende-se quase todos os critérios estabelecidos pelo parceiro, ou fica-se muito próximo de atendê-los.